In [2]:
from accounts import jp
# jp.ID, jp.PW

In [34]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium

from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [82]:
############################
# 크롬으로 잡플래닛 사이트 열기
############################

service = Service(executable_path="chromedriver.exe")
browser = webdriver.Chrome(service=service)

url = r"https://www.jobplanet.co.kr/job"

browser.maximize_window()
print(browser.get_window_size())

browser.get(r"https://www.jobplanet.co.kr/job")

{'width': 1512, 'height': 944}


In [83]:
############################
# 잡플래닛에 로그인 하기
############################

# 로그인 버튼 찾기 및 클릭
login_btn = browser.find_element(By.CSS_SELECTOR,"a.btn_txt.login")
login_btn.click()

# 로그인 ID와 PW 쳐야할 곳 클릭 + 적기
login_Id = browser.find_element(By.CSS_SELECTOR, "#user_email")
login_Id.click()
login_Id.send_keys(jp.ID)

login_Pw = browser.find_element(By.CSS_SELECTOR, "#user_password")
login_Pw.click()
login_Pw.send_keys(jp.PW)

# 이메일로 로그인 버튼 누르기
em_log = browser.find_element(By.CSS_SELECTOR, "fieldset > button")
em_log.click()

In [84]:
###########################
# 잡플래닛 검색창에 회사 입력하고 맨 처음에 나오는 기업 클릭하기.
###########################
# 검색창 누르기
search_bar = browser.find_element(By.CSS_SELECTOR, "#search_bar_search_query")
search_bar.click()

# 내가 원하는 회사 입력하기
want_cp = input()
search_bar.send_keys(want_cp)
search_bar.send_keys(Keys.RETURN)

# 검색결과 기다리기.
time.sleep(1)

# 맨처음에 나오는 기업 클릭하기
company = browser.find_element(By.CSS_SELECTOR, "div.is_company_card > div:nth-child(1) > a")
company.click()

In [85]:
###########################
# 회사페이지에서 리뷰 버튼 누르기
###########################
# 팝업창이 있을 때는 팝업창 버튼을 누르고 리뷰버튼을 누르기
try:
    pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
    pop.click()
    review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
    review_bar.click()

except NoSuchElementException:
    # 만약 팝업창 버튼이 없다면 리뷰 버튼만 누르기
    try:
        review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
        review_bar.click()
        time.sleep(1)
        pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
        pop.click()
    except NoSuchElementException:
        pass
    pass

In [86]:
###########################
# 회사페이지에서 리뷰 가져오기.
###########################
def get_jp_reviews():
    """
    잡플래닛 회사 리뷰 상세 내용을 url을 받아서 내용을 반환.
    연결
    """
    # 리뷰들을 리스트로 저장하는 변수
    review_list = []

    # 이전페이지 변수
    pre_page = 0

    # 무한 루프를 돌기.
    while True:

        # 소스 끌어오기.
        html = browser.page_source
        soup = BeautifulSoup(html, "lxml")

        # 현재 페이지의 숫자를 가져오기.
        crt_btn = soup.select_one('#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > strong')
        crt_page = int(crt_btn.text.strip())

        # 다음버튼을 눌렀은데도 같은 페이지인 경우
        if crt_page == pre_page:
            break

        # html 내용 가져오기
        review = soup.select("div.us_label_wrap > h2")

        # 텍스트로 추출한 내용을 리스트에 추가해서 넣어주기 위해 이용. (데이터 전처리)
        for text in review : # for문은 for 채워져있는 값의 원소 in 채워져있는 리스트
            text_clean_1 = text.get_text().strip()
            text_clean_2 = text_clean_1.replace("\n\n", "").replace('"', '').replace("T","").replace("\n","").replace(text_clean_1[:3], "").replace("      ","")
            review_list.append(text_clean_2)
        time.sleep(1)

        # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신하기.
            pre_page = crt_page

            # 다음 버튼 찾기
            next_btn = browser.find_element(By.CSS_SELECTOR,f"a.btn_pgnext")

            # 버튼이 있으면 클릭합니다.
            next_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(3)

        except:
            break

        # local에 디렉토리를 생성.
        save_path = f"/Users/gyeonmunju/Desktop/PlayData/jp_company"
        os.makedirs(save_path, exist_ok=True)

        # 데이터프레임으로 관리하겠다 지정. (데이터 생성하기.)
        jp_df = pd.DataFrame(review_list, columns=[f"jp_review"])

        # csv 파일로 저장.
        file_name = f"{want_cp}reviews.csv"
        save_file_path = os.path.join(save_path, file_name)
        jp_df.to_csv(save_file_path, index=False, encoding = "utf-8")

    return jp_df

# 함수 호출 후 결과 출력
get_jp_reviews()

# 브라우저 끄기
browser.close()

In [81]:
df1 = pd.read_csv("/Users/gyeonmunju/Desktop/PlayData/jp_company/이그니스reviews.csv")
df1

,jp_review
0,너무 급성장 했는지 아직까지도 갖춰진 체계는 없는데 뭔가를 많이 하려고 하는 회사.
1,모든게 금방 바뀌는 곳 한가지가 오래 가는 일이 없음
2,빠른 성장이 이루어지는 곳. 그러나 이를 감당할 리더의 부재.
3,브랜드 인지도 높고 희소성이 있어 대부분의 사람들이 알고 있음. 단백질 시장에서 자...
4,자율적인 분위기에 고속 성장을 이뤄나가고 있는 식품 스타트업
...,...
70,알바당시엔 굉장히 괜찮은 곳이었어요 직원분들도 다 좋아보였구요
71,자기만의 시간을 가지고 싶거나 워라밸이 중요하다면 추천하지않아요
72,겉보기와는 달리 매우 보수적인 회사. 야근을 강요하는 분위기. 그렇다고 효율적인 야...
73,랩노쉬 이미지같은 곳을 기대했다면 실망할것 투자나 외부의견에 영향을 많이 받는다


In [10]:
from accounts import jp
# jp.ID, jp.PW

import os
import time
import requests
import re
import pandas as pd
import selenium

from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

def get_jp_reviews(company_name):
    ############################
    # 크롬으로 잡플래닛 사이트 열기
    ############################
    # headless 모드

    option = webdriver.ChromeOptions()
    option.add_argument("--headless")

    service = Service(executable_path='chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=option)

    url = r"https://www.jobplanet.co.kr/job"

    browser.maximize_window()
    print(browser.get_window_size())

    browser.get(url)



    ############################
    # 잡플래닛에 로그인 하기
    ############################

    # 로그인 버튼 찾기 및 클릭
    login_btn = browser.find_element(By.CSS_SELECTOR,"a.btn_txt.login")
    login_btn.click()

    # 로그인 ID와 PW 쳐야할 곳 클릭 + 적기
    login_Id = browser.find_element(By.CSS_SELECTOR, "#user_email")
    login_Id.click()
    login_Id.send_keys(jp.ID)

    login_Pw = browser.find_element(By.CSS_SELECTOR, "#user_password")
    login_Pw.click()
    login_Pw.send_keys(jp.PW)

    # 이메일로 로그인 버튼 누르기
    em_log = browser.find_element(By.CSS_SELECTOR, "fieldset > button")
    em_log.click()
    time.sleep(2)



    ###########################
    # 잡플래닛 검색창에 회사 입력하고 맨 처음에 나오는 기업 클릭하기.
    ###########################
    # 검색창 누르기
    search_bar = browser.find_element(By.CSS_SELECTOR, "#search_bar_search_query")
    search_bar.click()

    # 내가 원하는 회사 입력하기
    search_bar.send_keys(company_name)
    search_bar.send_keys(Keys.RETURN)

    # 검색결과 기다리기.
    time.sleep(1)

    # 맨처음에 나오는 기업 클릭하기
    company = browser.find_element(By.CSS_SELECTOR, "div.is_company_card > div:nth-child(1) > a")
    company.click()

    # 검색결과 기다리기.
    time.sleep(1)



    ###########################
    # 회사페이지에서 리뷰 버튼 누르기
    ###########################
    # 팝업창이 있을 때는 팝업창 버튼을 누르고 리뷰버튼을 누르기
    try:
        pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
        pop.click()
        review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
        review_bar.click()

    except NoSuchElementException:
        # 만약 팝업창 버튼이 없다면 리뷰 버튼만 누르기
        try:
            review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
            review_bar.click()
            time.sleep(1)
            pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
            pop.click()
        except NoSuchElementException:
            pass
        pass



    ###########################
    # 회사페이지에서 리뷰 가져오기.
    ###########################
    """
    잡플래닛 회사 리뷰 상세 내용을 url을 받아서 내용을 반환.
    연결
    """
    # 리뷰들을 리스트로 저장하는 변수
    review_list = []

    # 이전페이지 변수
    pre_page = 0

    # 무한 루프를 돌기.
    while True:

        # 소스 끌어오기.
        html = browser.page_source
        soup = BeautifulSoup(html, "lxml")

        # 현재 페이지의 숫자를 가져오기.
        crt_btn = soup.select_one('#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > strong')
        crt_page = int(crt_btn.text.strip())

        # 다음버튼을 눌렀은데도 같은 페이지인 경우
        if crt_page == pre_page:
            break

        # html 내용 가져오기
        review = soup.select("div.us_label_wrap > h2")

        # 텍스트로 추출한 내용을 리스트에 추가해서 넣어주기 위해 이용. (데이터 전처리)
        for text in review : # for문은 for 채워져있는 값의 원소 in 채워져있는 리스트
            text_clean_1 = text.get_text().strip()
            text_clean_2 = text_clean_1.replace("\n\n", "").replace('"', '').replace("T","").replace("\n","").replace(text_clean_1[:3], "").replace("      ","")
            review_list.append(text_clean_2)
        time.sleep(1)

        # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신하기.
            pre_page = crt_page

            # 다음 버튼 찾기
            next_btn = browser.find_element(By.CSS_SELECTOR,f"a.btn_pgnext")

            # 버튼이 있으면 클릭합니다.
            next_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(3)

        except:
            break

        # local에 디렉토리를 생성.
        save_path = f"/Users/gyeonmunju/Desktop/PlayData/jp_company"
        os.makedirs(save_path, exist_ok=True)

        # 데이터프레임으로 관리하겠다 지정. (데이터 생성하기.)
        jp_df = pd.DataFrame(review_list, columns=[f"jp_review"])

        # csv 파일로 저장.
        file_name = f"{company_name}reviews.csv"
        save_file_path = os.path.join(save_path, file_name)
        jp_df.to_csv(save_file_path, index=False, encoding = "utf-8")
    
    # 브라우저 끄기
    browser.close()
    
    return jp_df

In [11]:
get_jp_reviews("제너레잇")

{'width': 800, 'height': 600}


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: headless chrome=114.0.5735.198); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102fa3f48 chromedriver + 4226888
1   chromedriver                        0x0000000102f9c4f4 chromedriver + 4195572
2   chromedriver                        0x0000000102be0d68 chromedriver + 281960
3   chromedriver                        0x0000000102bee36c chromedriver + 336748
4   chromedriver                        0x0000000102be5a78 chromedriver + 301688
5   chromedriver                        0x0000000102be462c chromedriver + 296492
6   chromedriver                        0x0000000102be6fe4 chromedriver + 307172
7   chromedriver                        0x0000000102be70d4 chromedriver + 307412
8   chromedriver                        0x000